# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> Data Science 2: Advanced Topics in Data Science 
## Homework 3 - Bayesian Analysis



**Harvard University**<br/>
**Spring 2020**<br/>
**Instructors**: Mark Glickman, Pavlos Protopapas, & Chris Tanner 


<hr style="height:2pt">

### Homework 3 is due March 5th

In [1]:
#PLEASE RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### INSTRUCTIONS

- To submit your assignment, please follow the instructions on Canvas.
- Please restart the kernel and run the entire notebook again before you submit.

<hr style="height:2pt">

### Please use the libraries below:

In [2]:
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
%matplotlib inline 
from datetime import datetime as dt
from scipy.special import expit


<hr style="height:2pt">

### Contraceptive Usage by Bangladeshi Women

In the second part of this assignment, you are provided with data sets `dataset_2_train.csv` and `dataset_2_test.csv`containing details of contraceptive usage by 1934 Bangladeshi women. There are four attributes for each woman, along with a label indicating if she uses contraceptives. The attributes include
* district: identifying code for the district the woman lives in
* urban: type of region of residence
* living.children: number of living children
* age-mean: age of the woman (in years, centered around mean)

The women are grouped into 60 districts. The task is to build a classification model that can predict if a given woman uses contraceptives.


### Part A: Bayesian Logistic Regression

The first model we will fit to the contraceptives data is a varying-intercept logistic regression model, where the intercept varies by district.

Prior distribution:

$\beta_{0j} \sim N(\mu_0,\sigma_0^2),$ with $\mu_0 \sim N(0,10000)$ and $\frac{1}{\sigma^2_0} \sim \text{Gamma}(0.1,0.1)$.

$\beta_1 \sim N(0,10000)$, $\beta_2\sim N(0,10000)$, $\beta_3 \sim N(0,10000)$

Model for data:

$Y_{ij} \sim \text{Bernoulli}(p_{ij})$

$\text{logit}\:p_{ij} = \beta_{0j} + \beta_1 \times \text{urban} + \beta_2 \times \text{living.children} + \beta_3 \times \text{age-mean}$,

where $Y_{ij}$ is 1 if woman $i$ in district $j$ uses contraceptives, and 0 otherwise, and where $i \in \{1,...,N\}$ and $j \in \{1,...,J\}$. $N$ is the number of observations in the data, and $J$ is the number of districts. The above notation assumes $N(\mu, \sigma^2)$ is a Normal distribution with mean $\mu$ and variance $\sigma^2$.

##### **A1**

As a preprocessing step, map the district number 61 to the number 54 so that the districts are in order. Also, re-name district 60 to be district 0 so that the districts are numbered 0 through 59. **We use this numbering throughout the homework, and will grade using these district numbers**

In [3]:
#your code here

train = pd.read_csv('data/dataset_2_train.csv')
test = pd.read_csv('data/dataset_2_test.csv')

train.loc[train.district==61,'district'] = 54
test.loc[test.district==61,'district'] = 54

train.loc[train.district==60,'district'] = 0
test.loc[test.district==60,'district'] = 0

display(train.head())
display(train.describe())
print("Training data size = {}".format(train.shape))
print("Testing data size = {}".format(test.shape))

,district,urban,living.children,age_mean,contraceptive_use
0,35,0,4,2.4400,0
1,22,0,2,-1.5599,1
2,29,0,2,-8.5599,1
3,5,0,3,-4.5599,1
4,34,1,4,8.4400,0


,district,urban,living.children,age_mean,contraceptive_use
count,967.000000,967.000000,967.000000,967.000000,967.000000
mean,28.707342,0.307135,2.591520,-0.310626,0.389866
std,17.408268,0.461545,1.248457,8.995747,0.487972
min,0.000000,0.000000,1.000000,-13.560000,0.000000
25%,14.000000,0.000000,1.000000,-7.560000,0.000000
50%,29.000000,0.000000,3.000000,-1.559900,0.000000
75%,45.000000,1.000000,4.000000,6.440000,1.000000
max,59.000000,1.000000,4.000000,19.440000,1.000000


Training data size = (967, 5)
Testing data size = (967, 5)


##### **A2**

First, we'll verify that pymc3 can recover the hidden parameter values. To do so, we'll hard-code known values and simulate data from the model, then using that data we'll check if pymc3 can get back the parameters we hard-coded. If it does, we'll have hope that it can get the hidden parameter values that generated the real data.

Use the following hard-coded values:  
$\mu_0 = 2$

$\sigma^2_0 = 1$

$\beta_{0j} \sim N(\mu_0,\sigma_0^2)$ for all sixty districts

$\beta_1 = 4$

$\beta_2 = -3$

$\beta_3 = -2$

**Verify that these parameters give an overall average of 46 to 48 percent of subjects using contraceptives**

In [4]:
#your code here

# setting seed for replication
np.random.seed(109)

# hard-coding of parameters
j = train.district.nunique() # number of Beta_0's
mu_0 = 2
sigma_0 = 1
beta_0 = np.random.normal(mu_0, sigma_0, j) 
print("\nBeta_0:\n{}\n".format(beta_0))

beta_1 = 4
beta_2 = -3
beta_3 = -2

# Generate simulated data
simulated_df = train.copy()
simulated_df['contraceptive_use'] = np.random.binomial(1, expit(beta_0[simulated_df['district']] + 
                                                                beta_1*simulated_df['urban'] + 
                                                                beta_2*simulated_df['living.children'] + 
                                                                beta_3*simulated_df['age_mean']
                                                               ))

display(simulated_df.head())
display(simulated_df.describe())

prop = simulated_df['contraceptive_use'].mean()*100
print ("\nProportion of women using contraceptives = \t{0:.2f}%\n".format(prop))


Beta_0:
[1.81226506 4.44063677 1.05909216 2.61391165 2.73562667 2.97291789
 2.25919495 1.92592508 1.82231198 1.16125645 1.84371288 1.65823837
 2.81873029 1.90738606 2.50808094 0.2457775  1.63518202 1.96908145
 1.87385573 2.299181   3.82207405 3.41717686 1.54268378 2.86019957
 2.28468256 2.51062749 3.57829456 2.16130871 2.71819658 1.19400416
 1.40181919 3.32383635 3.13726714 1.51166981 2.34806743 1.02913865
 1.78786357 2.42335106 2.10377004 2.05097481 1.32855935 2.27170251
 2.00701349 2.32662299 2.03618145 0.80672159 1.84189829 2.01341776
 1.52765319 1.79189355 3.44154411 0.32890473 2.54455933 3.26132175
 2.96283701 1.72009832 1.76955224 2.45213427 1.64578923 2.34015305]



,district,urban,living.children,age_mean,contraceptive_use
0,35,0,4,2.4400,0
1,22,0,2,-1.5599,0
2,29,0,2,-8.5599,1
3,5,0,3,-4.5599,1
4,34,1,4,8.4400,0


,district,urban,living.children,age_mean,contraceptive_use
count,967.000000,967.000000,967.000000,967.000000,967.000000
mean,28.707342,0.307135,2.591520,-0.310626,0.476732
std,17.408268,0.461545,1.248457,8.995747,0.499717
min,0.000000,0.000000,1.000000,-13.560000,0.000000
25%,14.000000,0.000000,1.000000,-7.560000,0.000000
50%,29.000000,0.000000,3.000000,-1.559900,0.000000
75%,45.000000,1.000000,4.000000,6.440000,1.000000
max,59.000000,1.000000,4.000000,19.440000,1.000000



Proportion of women using contraceptives = 	47.67%



##### **A3**

Fit the varying-intercept model specified above to your simulated data. Reasonably good convergence may require adjustments to the number of tuning samples and the target acceptance rate as suggested in pymc3's warnings.

Note: It is wise to take advantage of a model's built-in `summary` method to inspect it after fitting

In [ ]:
#your code here

# setting seed for replication

for i in range(5):
    for k in range(10):
    
        np.random.seed(109)

        with pm.Model() as model:
            # prior
            mu_0 = pm.Normal('mu_0', mu=0, sd=100)
            tau_beta_0 = pm.Gamma('tau_beta_0', 0.1, 0.1)
            beta_0 = pm.Normal('beta_0', mu=mu_0, tau=tau_beta_0, shape=j)
            beta_1 = pm.Normal('beta_1', mu=0, sd=100)
            beta_2 = pm.Normal('beta_2', mu=0, sd=100)
            beta_3 = pm.Normal('beta_3', mu=0, sd=100)

            # likelihood
            alpha = beta_0[simulated_df['district'].values]

            p_i = pm.Deterministic('$P_i$', pm.math.invlogit(alpha +
                                                             (beta_1 * simulated_df['urban']) + 
                                                             (beta_2 * simulated_df['living.children']) + 
                                                             (beta_3 * simulated_df['age_mean'])
                                                            ))

            obs = pm.Bernoulli(name='c_use', p=p_i, observed=simulated_df['contraceptive_use'])
            trace = pm.sample(1000 + 100*(k), tune=2000, target_accept=0.95)
        
        display(pm.summary(trace, var_names=['mu_0', 'tau_beta_0', 'beta_1', 'beta_2', 'beta_3']))

/usr/share/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [beta_3, beta_2, beta_1, beta_0, tau_beta_0, mu_0]
Sampling chain 1, 0 divergences:  34%|███▍      | 1020/3000 [04:38<09:12,  3.58it/s] 

In [30]:
for i in range(20):
    for k in range(20):
        display(pm.summary(trace[i][j], var_names=['mu_0', 'tau_beta_0', 'beta_1', 'beta_2', 'beta_3']))
#display(pm.summary(trace, var_names=['beta_0']))

/usr/share/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu_0,-1.565,0.222,-1.889,-1.132,0.030,0.021,54.0,54.0,47.0,117.0,1.04
tau_beta_0,4.162,2.020,1.737,7.822,0.250,0.178,65.0,65.0,70.0,158.0,1.01
beta_1,0.735,0.161,0.389,1.014,0.011,0.008,224.0,224.0,232.0,146.0,1.00
beta_2,0.310,0.070,0.185,0.427,0.009,0.006,65.0,65.0,62.0,137.0,1.03
beta_3,-0.033,0.011,-0.054,-0.014,0.001,0.001,156.0,156.0,157.0,238.0,1.02


/usr/share/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
beta_0[0],-2.018,0.450,-2.877,-1.140,0.036,0.026,158.0,151.0,167.0,245.0,1.01
beta_0[1],-2.084,0.341,-2.669,-1.386,0.031,0.022,119.0,119.0,119.0,212.0,1.02
beta_0[2],-1.705,0.457,-2.680,-0.933,0.028,0.020,266.0,266.0,276.0,324.0,1.01
beta_0[3],-1.461,0.528,-2.393,-0.395,0.038,0.027,192.0,192.0,198.0,218.0,1.01
beta_0[4],-1.389,0.458,-2.275,-0.627,0.033,0.025,189.0,175.0,191.0,255.0,1.00
beta_0[5],-1.557,0.446,-2.327,-0.678,0.045,0.032,97.0,95.0,95.0,186.0,1.02
beta_0[6],-1.912,0.389,-2.553,-1.071,0.032,0.022,152.0,152.0,154.0,226.0,1.01
beta_0[7],-1.964,0.459,-2.834,-1.069,0.037,0.026,157.0,152.0,167.0,232.0,1.01
beta_0[8],-1.342,0.433,-2.161,-0.578,0.030,0.022,207.0,198.0,211.0,269.0,1.03
beta_0[9],-2.001,0.476,-2.973,-1.195,0.039,0.027,151.0,151.0,147.0,171.0,1.03


##### **A4**

Plot the trace plots of the MCMC sampler for the parameters $\mu_0, \sigma_0, \beta_1, \beta_2, \beta_3$. Based on these and the R-hat values, does it look like the samplers converged?

In [ ]:
#your code here


your response here

##### **A5**

Plot histograms of the posterior distributions for the parameters $\beta_{0,9}$, $\beta_{0,19}$, $\beta_{0,29}$, ..., $\beta_{0,59}$. Are the actual parameters that you generated contained within these posterior distributions? 

In [ ]:
#your code here


your response here


##### **A6**

We now fit our model to the actual data. Fit the varying-intercept model to the real training data.

In [ ]:
#your code here


##### **A7**

Check the convergence by examining the trace plots and R-hats, as you did with the simulated data. What do you observe?

In [ ]:
#your code here


your response here

##### **A8**

Based on the posterior means, which district has the highest base rate of contraceptive usage (independent of other factors like urban population)? Which district has the lowest?

In [ ]:
#your code here


your response here

##### **A9**

What are the posterior means of $\mu_0$ and $\sigma^2_0$? Do these values offer any evidence in support of or against the varying-intercept model, compared to a model with a single intercept value for all districts or a model with an intercept of 0 for all districts?

In [ ]:
#your code here


your response here

### Part B: Varying-Coefficients Model

The next model we will fit to the contraceptives data is a varying-coefficients logistic regression model, where the coefficients on living.children, age-mean, and urban vary by district.

$\beta_{0j} \sim N(\mu_0,\sigma_0^2),$ with $\mu_0 \sim N(0,10000)$ and $\frac{1}{\sigma_0^2} \sim \text{Gamma}(0.1,0.1)$.

$\beta_{1j} \sim N(0,\sigma_1^2)$, with $\frac{1}{\sigma^2_1} \sim \text{Gamma}(0.1,0.1)$

$\beta_{2j}\sim N(0,\sigma_2^2)$, with $\frac{1}{\sigma^2_2}\sim \text{Gamma}(0.1,0.1)$

$\beta_{3j} \sim N(0,\sigma_3^2)$, with $\frac{1}{\sigma^2_3} \sim \text{Gamma}(0.1,0.1)$

Model for data:

$Y_{ij} \sim \text{Bernoulli}(p_{ij})$

$\text{logit}\:p_{ij} = \beta_{0j} + \beta_{1j} \times \text{urban} + \beta_{2j} \times \text{living.children} + \beta_{3j} \times \text{age-mean}$,

##### **B1**

Fit the model to the real data. Plot posterior means for each of the coefficients to the predictors. What do you conclude from these graphs?

In [ ]:
#your code here


your response here

##### **B2**

Use all of the information you've gleaned thus far to build a final Bayesian logistic regression classifier on the train set. Then, use your model to make predictions on the test set. Report your model's classification percentage.

In [ ]:
#your code here


your response here